# Machine Learning Project 02
## C4.5 Decision Tree Implementation
### Iris Dataset Classification

## Step 1: Import Libraries

In [133]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris  # Only for loading data
from sklearn.model_selection import train_test_split  # Only for splitting data
import math

## Step 2: Node Class

In [135]:
class Node:
    def __init__(self):
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.class_label = None
        self.is_leaf = False

## Step 3: DecisionTreeC4.5 Class

In [137]:
class DecisionTreeC45:
    
    def __init__(self, max_depth=None, min_samples_split=2):
        """
        Parameters:
        - max_depth: Maximum depth of the tree (None = no limit)
        - min_samples_split: Minimum number of samples required to split a node
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        self.n_classes = None
        self.n_features = None
    
    
    def entropy(self, y):
        # Count occurrences of each class
        unique_classes, counts = np.unique(y, return_counts=True)
        
        # Calculate probability of each class
        probabilities = counts / len(y)
        
        # Calculate entropy
        entropy_value = 0
        for p in probabilities:
            if p > 0:  # Avoid log(0)
                entropy_value -= p * math.log2(p)
        
        return entropy_value
    
    
    def information_gain_ratio(self, parent, left_child, right_child):
        """
        Calculate Information Gain Ratio (C4.5 improvement)
        
        Formulas:
            IG = Entropy(parent) - Weighted_Avg(Entropy(children))
            SI = Split Information
            IGR = IG / SI
        """
        # Calculate parent entropy
        entropy_parent = self.entropy(parent)
        
        # Number of samples
        n_total = len(parent)
        n_left = len(left_child)
        n_right = len(right_child)
        
        # Calculate children entropy
        entropy_left = self.entropy(left_child) if n_left > 0 else 0
        entropy_right = self.entropy(right_child) if n_right > 0 else 0
        
        # Calculate weighted average of children entropy
        entropy_weighted = (n_left / n_total) * entropy_left + (n_right / n_total) * entropy_right
        
        # Information Gain
        information_gain = entropy_parent - entropy_weighted
        
        # Split Information calculation
        p_left = n_left / n_total
        p_right = n_right / n_total
        
        split_information = 0
        if p_left > 0:
            split_information -= p_left * math.log2(p_left)
        if p_right > 0:
            split_information -= p_right * math.log2(p_right)
        
        # Avoid division by zero
        if split_information == 0:
            return information_gain
        
        # Information Gain Ratio
        gain_ratio = information_gain / split_information
        
        return gain_ratio
    
    
    def find_best_split(self, X, y):

        best_feature = None
        best_threshold = None
        best_gain_ratio = 0
        
        # Iterate through all features
        for feature_idx in range(X.shape[1]):
            feature_values = X[:, feature_idx]
            unique_values = np.unique(feature_values)
            
            # Try all possible thresholds
            for i in range(len(unique_values) - 1):
                # Threshold is the average of two consecutive values
                threshold = (unique_values[i] + unique_values[i + 1]) / 2
                
                # Split data based on this threshold
                left_mask = feature_values <= threshold
                right_mask = feature_values > threshold
                
                # Skip if split is invalid (one side is empty)
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue
                
                # Calculate Information Gain Ratio
                gain_ratio = self.information_gain_ratio(y, y[left_mask], y[right_mask])
                
                # Update best split if this is better
                if gain_ratio > best_gain_ratio:
                    best_gain_ratio = gain_ratio
                    best_feature = feature_idx
                    best_threshold = threshold
        
        return best_feature, best_threshold, best_gain_ratio
    
    
    def build_tree(self, X, y, depth=0):

        n_samples = len(y)
        n_classes = len(np.unique(y))
        
        # Stopping criterion 1: Pure (one class)
        if n_classes == 1:
            leaf = Node()
            leaf.class_label = y[0]
            leaf.is_leaf = True
            return leaf
        
        # Stopping criterion 2: Too few samples
        if n_samples < self.min_samples_split:
            leaf = Node()
            leaf.class_label = np.bincount(y).argmax()  # Majority class
            leaf.is_leaf = True
            return leaf
        
        # Stopping criterion 3: Maximum depth reached
        if self.max_depth is not None and depth >= self.max_depth:
            leaf = Node()
            leaf.class_label = np.bincount(y).argmax()
            leaf.is_leaf = True
            return leaf
        
        # Find the best split
        feature, threshold, gain_ratio = self.find_best_split(X, y)
        
        # Stopping criterion 4: No good split
        if feature is None or gain_ratio == 0:
            leaf = Node()
            leaf.class_label = np.bincount(y).argmax()
            leaf.is_leaf = True
            return leaf
        
        # Create decision node
        node = Node()
        node.feature = feature
        node.threshold = threshold
        
        # Split data
        left_mask = X[:, feature] <= threshold
        right_mask = X[:, feature] > threshold
        
        # Recursively build children
        node.left = self.build_tree(X[left_mask], y[left_mask], depth + 1)
        node.right = self.build_tree(X[right_mask], y[right_mask], depth + 1)
        
        return node
    
    
    def fit(self, X, y):
        """
        Train the decision tree on training data
        """
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.root = self.build_tree(X, y)
        return self
    
    
    def predict_single(self, x, node=None):
        """
        Predict class for a single sample
        """
        if node is None:
            node = self.root
        
        # If leaf node, return class label
        if node.is_leaf:
            return node.class_label
        
        # Traverse left or right based on threshold
        if x[node.feature] <= node.threshold:
            return self.predict_single(x, node.left)
        else:
            return self.predict_single(x, node.right)
    
    
    def predict(self, X):
        """
        Predict classes for multiple samples
        """
        predictions = np.array([self.predict_single(x) for x in X])
        return predictions

## Step 4: Load and Explore Data

In [139]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

print("Iris Dataset Overview")
print(f"\nNumber of samples: {len(X)}")
print(f"Number of features: {X.shape[1]}")
print(f"Number of classes: {len(iris.target_names)}")
print(f"Class names: {iris.target_names}")
print(f"\nFeature names:")
for i, name in enumerate(iris.feature_names, 1):
    print(f"  {i}. {name}")

Iris Dataset Overview

Number of samples: 150
Number of features: 4
Number of classes: 3
Class names: ['setosa' 'versicolor' 'virginica']

Feature names:
  1. sepal length (cm)
  2. sepal width (cm)
  3. petal length (cm)
  4. petal width (cm)


## Step 5: Split Data into Train and Test Sets

In [141]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nData split:")
print(f"  Train: {len(X_train)} samples")
print(f"  Test:  {len(X_test)} samples")


Data split:
  Train: 120 samples
  Test:  30 samples


## Step 6: Train the Model

In [143]:
# Create and train the model
model = DecisionTreeC45(max_depth=10, min_samples_split=2)
model.fit(X_train, y_train)

print("Model trained successfully!")

Model trained successfully!


## Step 7: Make Predictions and Evaluate

In [145]:
# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate accuracy
accuracy_train = np.mean(y_pred_train == y_train)
accuracy_test = np.mean(y_pred_test == y_test)

print("Prediction Results")
print(f"\nTrain Accuracy: {accuracy_train:.4f} ({accuracy_train*100:.2f}%)")
print(f"Test Accuracy:  {accuracy_test:.4f} ({accuracy_test*100:.2f}%)")

Prediction Results

Train Accuracy: 1.0000 (100.00%)
Test Accuracy:  0.9333 (93.33%)


## Step 8: Detailed Evaluation Metrics

In [147]:
def calculate_metrics(y_true, y_pred, n_classes, class_names):
    """
    Calculate evaluation metrics for each class
    """
    metrics = {}
    
    for class_idx in range(n_classes):
        # True Positives, False Positives, False Negatives, True Negatives
        tp = np.sum((y_pred == class_idx) & (y_true == class_idx))
        fp = np.sum((y_pred == class_idx) & (y_true != class_idx))
        fn = np.sum((y_pred != class_idx) & (y_true == class_idx))
        tn = np.sum((y_pred != class_idx) & (y_true != class_idx))
        
        # Calculate metrics
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics[class_names[class_idx]] = {
            'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        }
    
    return metrics

# Calculate metrics
metrics_test = calculate_metrics(y_test, y_pred_test, len(iris.target_names), iris.target_names)

print("Detailed Evaluation Metrics (Test Set)")

for class_name in iris.target_names:
    m = metrics_test[class_name]
    print(f"\nClass: {class_name}")
    print(f"   Precision: {m['Precision']:.4f}")
    print(f"   Recall:    {m['Recall']:.4f}")
    print(f"   F1-Score:  {m['F1-Score']:.4f}")

# Average metrics
avg_precision = np.mean([metrics_test[cn]['Precision'] for cn in iris.target_names])
avg_recall = np.mean([metrics_test[cn]['Recall'] for cn in iris.target_names])
avg_f1 = np.mean([metrics_test[cn]['F1-Score'] for cn in iris.target_names])

print("\nAverage Metrics (Macro Average):")
print(f"   Precision: {avg_precision:.4f}")
print(f"   Recall:    {avg_recall:.4f}")
print(f"   F1-Score:  {avg_f1:.4f}")

Detailed Evaluation Metrics (Test Set)

Class: setosa
   Precision: 1.0000
   Recall:    1.0000
   F1-Score:  1.0000

Class: versicolor
   Precision: 0.9000
   Recall:    0.9000
   F1-Score:  0.9000

Class: virginica
   Precision: 0.9000
   Recall:    0.9000
   F1-Score:  0.9000

Average Metrics (Macro Average):
   Precision: 0.9333
   Recall:    0.9333
   F1-Score:  0.9333


## Step 9: Confusion Matrix

In [149]:
def confusion_matrix(y_true, y_pred, n_classes):
    """
    Calculate Confusion Matrix
    """
    cm = np.zeros((n_classes, n_classes), dtype=int)
    
    for i in range(len(y_true)):
        actual = y_true[i]
        predicted = y_pred[i]
        cm[actual][predicted] += 1
    
    return cm

# Calculate Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test, len(iris.target_names))

print("Confusion Matrix (Test Set)")
print("\n" + " "*27 + "Predicted")
header = " "*15
for name in iris.target_names:
    header += f"{name:<12}"
print(header)
print("-"*60)

for i, class_name in enumerate(iris.target_names):
    row = f"Actual {class_name:<10}"
    for j in range(len(iris.target_names)):
        row += f"{cm[i, j]:<12}"
    print(row)

Confusion Matrix (Test Set)

                           Predicted
               setosa      versicolor  virginica   
------------------------------------------------------------
Actual setosa    10          0           0           
Actual versicolor0           9           1           
Actual virginica 0           1           9           
